In [ ]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'  # always print last expr.
%config InlineBackend.figure_format = 'svg'
%load_ext autoreload
%autoreload 2
%matplotlib inline

import logging

logging.basicConfig(level=logging.DEBUG)

In [ ]:
import torch

t0 = torch.tensor([0.0])
t = torch.sort(torch.rand(9)).values

In [ ]:
s = torch.rand(5)

In [ ]:
inds = torch.searchsorted(t, s, right=True)

In [ ]:
T = torch.concat([t0, t])

In [ ]:
t

In [ ]:
s

In [ ]:
T[inds]

In [ ]:
?torch.searchsorted